In [3]:
import os
import torchaudio
import numpy as np
import pandas as pd

def calculate_snr(mixed_waveform, noise_waveform):
    mixed_waveform = np.array(mixed_waveform)
    noise_waveform = np.array(noise_waveform)
    
    mixed_power = np.mean(mixed_waveform**2)
    noise_power = np.mean(noise_waveform**2)
    signal_power = mixed_power - noise_power

    #if signal_power <= 0:
        #raise ValueError("Signal power is too low or negative. Check your input waveforms.")
    if noise_power == 0:
        return np.inf
    snr = 10 * np.log10(signal_power / noise_power)
    return snr


bc_path = '/home/alessandrab/python_files/data_noisy/bc'
ac_path = '/home/alessandrab/python_files/data_noisy/ac'
noise_path = '/home/alessandrab/python_files/edited_noise'

bc_files = os.listdir(bc_path)
ac_files = os.listdir(ac_path)
noise_files = os.listdir(noise_path)

#AC
speech_files_ac = (f for f in os.listdir(ac_path) if f.endswith('.wav'))
noise_files_ac = (f for f in os.listdir(noise_path) if f.endswith('_2.wav'))
#print(speech_files_ac)
speech_list_ac = [(waveform.unsqueeze(0) if waveform.ndim == 1 else waveform, sample_rate, file) 
               for file in speech_files_ac 
               for waveform, sample_rate in [torchaudio.load(os.path.join(ac_path, file))]]
 
noise_list_ac = [(waveform.unsqueeze(0) if waveform.ndim == 1 else waveform, sample_rate, file) 
              for file in noise_files_ac
              for waveform, sample_rate in [torchaudio.load(os.path.join(noise_path, file))]]

#BC
speech_files_bc = (f for f in os.listdir(bc_path) if f.endswith('.wav'))
noise_files_bc = (f for f in os.listdir(noise_path) if f.endswith('_1.wav'))

speech_list_bc = [(waveform.unsqueeze(0) if waveform.ndim == 1 else waveform, sample_rate, file) 
               for file in speech_files_bc
               for waveform, sample_rate in [torchaudio.load(os.path.join(bc_path, file))]]

noise_list_bc = [(waveform.unsqueeze(0) if waveform.ndim == 1 else waveform, sample_rate, file) 
              for file in noise_files_bc
              for waveform, sample_rate in [torchaudio.load(os.path.join(noise_path, file))]]

df = pd.DataFrame(columns=['AC Name', 'AC Noise Name', 'SNR AC', 'BC Name', 'BC Noise Name', 'SNR BC', 'SNR Difference'])

for wf_ac_speech, sr_ac_speech, speech_name_ac in speech_list_ac:
    for wf_ac_noise, sr_ac_noise, noise_name_ac in noise_list_ac:
        if speech_name_ac.split('_')[4]==noise_name_ac.split('_')[1] and speech_name_ac.split('_')[5]==noise_name_ac.split('_')[2]:
            for wf_bc_speech, sr_bc_speech, speech_name_bc in speech_list_bc:
             if speech_name_ac.split('_')[0]==speech_name_bc.split('_')[0] and speech_name_ac.split('_')[1]==speech_name_bc.split('_')[1] and speech_name_ac.split('_')[3]==speech_name_bc.split('_')[3] and speech_name_ac.split('_')[4]==speech_name_bc.split('_')[4]:
                for wf_bc_noise, sr_bc_noise, noise_name_bc in noise_list_bc:
                    if speech_name_bc.split('_')[4]==noise_name_bc.split('_')[1] and speech_name_bc.split('_')[5]==noise_name_bc.split('_')[2]:
                        snr_ac = calculate_snr(wf_ac_speech, wf_ac_noise)
                        snr_bc = calculate_snr(wf_bc_speech, wf_bc_noise)
                        diff = snr_ac - snr_bc
                        new_row = pd.DataFrame([[speech_name_ac, noise_name_ac, snr_ac, speech_name_bc, noise_name_bc, snr_bc, diff]], columns=['AC Name', 'AC Noise Name', 'SNR AC', 'BC Name', 'BC Noise Name', 'SNR BC', 'SNR Difference'])
                        df = pd.concat([df, new_row], ignore_index=True)






/home/alessandrab/tmp/ipykernel_19317/1689135654.py:18: RuntimeWarning: invalid value encountered in log10
  snr = 10 * np.log10(signal_power / noise_power)


In [4]:
df.head()

,AC Name,AC Noise Name,SNR AC,BC Name,BC Noise Name,SNR BC,SNR Difference
0,MONSSEF_3_2_NOISE_24_2.wav,NOISE_3_2.wav,6.423227,MONSSEF_3_1_NOISE_24_1.wav,NOISE_22_1.wav,22.276459,-15.853232
1,MONSSEF_3_2_NOISE_24_2.wav,NOISE_3_2.wav,6.423227,MONSSEF_3_1_NOISE_24_1.wav,NOISE_4_1.wav,19.048877,-12.625650
2,MONSSEF_3_2_NOISE_24_2.wav,NOISE_3_2.wav,6.423227,MONSSEF_3_1_NOISE_24_1.wav,NOISE_34_1.wav,20.403144,-13.979917
3,MONSSEF_3_2_NOISE_24_2.wav,NOISE_3_2.wav,6.423227,MONSSEF_3_1_NOISE_24_1.wav,NOISE_3_1.wav,20.426192,-14.002965
4,MONSSEF_3_2_NOISE_24_2.wav,NOISE_3_2.wav,6.423227,MONSSEF_3_1_NOISE_24_1.wav,NOISE_12_1.wav,10.880295,-4.457068


In [5]:
output_csv_path = "/home/alessandrab/python_files/data_noisy/snr_results.csv"
df.to_csv(output_csv_path, index=False)